In [3]:
#!/usr/bin/env python3
"""
Create a rotated MRS voxel mask in T1 space by testing T1 voxel world coords
in MRS voxel-index coordinates (handles rotation/tilt encoded in MRS affine).

Usage: edit filenames below and run:
    python make_mrs_mask_rotated.py
"""

import numpy as np
import nibabel as nib
import os

os.chdir("/Volumes/vdrive/helpern_users/helpern_j/IAM/IAM_Analysis/IAM_BIDS/derivatives/mrs/04_pc_voxel_segmentation/1002")

# -------------------------
# User inputs - edit these
# -------------------------
t1_file  = "T1.nii.gz"            # your T1 (target) image
mrs_file = "PC_PRESS.nii.gz"     # MRS 1x1x1 reference (affine contains geometry)
out_file = "MRS_T1_mask.nii.gz"

# origin: 'corner' means voxel indices occupy [0,1] along each axis (index 0 is corner)
#         'center' means voxel indices occupy [-0.5, +0.5] around the center
# Choose 'corner' if your earlier yellow ([0,0,0]) matched better; otherwise try 'center'.
origin = 'center'  # options: 'corner' or 'center'

# optionally keep a tiny tolerance (mm in index units) - helps with float rounding
tol = 1e-6

# -------------------------
# Load images & metadata
# -------------------------
t1 = nib.load(t1_file)
t1_affine = t1.affine
t1_shape = t1.shape

mrs = nib.load(mrs_file)
mrs_affine = mrs.affine
mrs_zoom = np.array(mrs.header.get_zooms()[:3])  # should be ~20,20,20 for you

print("T1 shape:", t1_shape)
print("MRS voxel zooms (mm):", mrs_zoom)
print("MRS affine:\n", mrs_affine)

# Precompute inverse affine that maps world -> MRS voxel-index coords
inv_mrs_affine = np.linalg.inv(mrs_affine)

# Define index-space inclusion bounds depending on origin convention
if origin == 'corner':
    lo = 0.0 - tol
    hi = 1.0 + tol
elif origin == 'center':
    lo = -0.5 - tol
    hi = 0.5 + tol
else:
    raise ValueError("origin must be 'corner' or 'center'")

# -------------------------
# Build mask
# -------------------------
# Generate grid of T1 voxel indices
i, j, k = np.indices(t1_shape)
nvox = i.size
ijk = np.vstack([i.ravel(), j.ravel(), k.ravel(), np.ones(nvox)])  # shape (4, N)

# Convert T1 voxel indices -> world coords using T1 affine
world = t1_affine @ ijk   # shape (4, N)
world_xyz = world[:3, :]  # (3, N)

# Convert world coords -> MRS voxel-index coordinates using inverse MRS affine
# For each world point w, idx = inv_mrs_affine @ [wx, wy, wz, 1]
mrs_idx = inv_mrs_affine @ np.vstack([world_xyz, np.ones(nvox)])  # (4, N)
mrs_idx_xyz = mrs_idx[:3, :]

# Check inclusion: all three index coords must be between lo and hi
inside_mask = (
    (mrs_idx_xyz[0, :] >= lo) & (mrs_idx_xyz[0, :] <= hi) &
    (mrs_idx_xyz[1, :] >= lo) & (mrs_idx_xyz[1, :] <= hi) &
    (mrs_idx_xyz[2, :] >= lo) & (mrs_idx_xyz[2, :] <= hi)
)

mask = inside_mask.reshape(t1_shape).astype(np.uint8)

# Save mask using T1 affine (so it displays correctly on T1)
mask_img = nib.Nifti1Image(mask, t1_affine)
nib.save(mask_img, out_file)

print("Saved rotated mask:", out_file)
print("Mask voxels (count):", int(mask.sum()))
print("Origin convention used:", origin)


T1 shape: (192, 256, 256)
MRS voxel zooms (mm): [20. 20. 20.]
MRS affine:
 [[ 19.97452955   0.90374036   0.44878073   2.416907  ]
 [  1.00851947 -17.59422694  -9.45653662 -20.645858  ]
 [ -0.03251998   9.46712373 -17.61739229  11.55861   ]
 [  0.           0.           0.           1.        ]]
Saved rotated mask: MRS_T1_mask_rotated_in_T12.nii.gz
Mask voxels (count): 8000
Origin convention used: center
